# Problem 303: Multiples with Small Digits

For a positive integer $n$, define $f(n)$ as the least positive multiple of $n$ that, written in base $10$, uses only digits $\le 2$.

Thus $f(2)=2$, $f(3)=12$, $f(7)=21$, $f(42)=210$, $f(89)=1121222$.

Also, 

$$
\sum_{n=1}^{100} {f(n) \over n} = 11363107
$$

Find
$$
\sum_{n=1}^{10000} {f(n) \over n}
$$

First steps, starting relatively simply:

In [1]:
import library

In [2]:
def f(n):
    k = n
    while set(library.digits(k)) - {0, 1, 2} != set():
        k += n

    return k

Run some tests to confirm the examples in the problem:

In [3]:
f(2)

2

In [4]:
f(3)

12

In [5]:
f(7)

21

In [6]:
f(42)

210

In [7]:
f(89)

1121222

In [8]:
sum([ f(n) / n for n in range(1, 101) ])

11363107.0

Now give the full sum a try.  _(You probably wouldn't expect this to work ... and it doesn't.)_

In [9]:
# sum([ f(n) / n for n in range(1, 10001) ])

## Performance Testing

As usual, my initial implementation was pretty slow.  I'm not sure how far it got before I killed it.  Let me try some speed testing to see if there's something better:

In [10]:
# Starting point
def f(n):
    k = n
    while set(library.digits(k)) - {0, 1, 2} != set():
        k += n

    return k

In [11]:
%%timeit
x = 1234
library.digits(x)

374 ns ± 5.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
%%timeit
x = 1234
set(library.digits(x))

492 ns ± 22.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%timeit
x = 1234
set(library.digits(x)) - {0,1,2}

558 ns ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [14]:
%%timeit
x = 1234
set(library.digits(x)) - {0,1,2} != set()

617 ns ± 27.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [15]:
%%timeit
x = 1234
str(x)

49.2 ns ± 3.9 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [16]:
%%timeit
# instead of building a set of all digits, just loop through the digits and break on the first one greater than two:
x = 1234
for x in str(x):
    if int(x) > 2:
        break
else:
    True

266 ns ± 24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [17]:
%%timeit
# same approach as above, but without doing an integer conversion:
x = 1234
digits = {'0', '1', '2'}
for x in str(x):
    if x not in digits:
        break
else:
    True

162 ns ± 9.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Conclusion: we can improve performance substantially by not using `set`, and by returning a result after the first digit that isn't in $\{0, 1, 2\}$.

In [18]:
def digit_greater_than_two(n):
    digits = {'0', '1', '2'}
    for d in str(n):
        if d not in digits:
            break
    else:
        return True

    return False

In [19]:
def g(n):
    k = n
    while not digit_greater_than_two(k):
        k += n

    return k

In [20]:
%%timeit
f(89)

12.1 ms ± 997 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit
g(89)

2.16 ms ± 76.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
g(89)

1121222

I didn't test it, but I suspect this approach also scales better to larger numbers.

Try the full sum again. _(I liked my chances better this time ... but this still doesn't work.)_

In [23]:
# sum([ g(n) / n for n in range(1, 10001) ])

## Analysis and Planning

Letting the script run using $g(n)$ didn't produce a total in a reasonable amount of time.  Some "napkin" calculations suggest that this could take a _very_ long time.

If I am really getting ~5ms per execution (as I did with g(89) above), this shouldn't take too long.  Estimated completion would be under a minute:

In [24]:
0.005 * 10000 / 60

0.8333333333333334

Total run time (in hours) with 30 seconds per iteration is less comforting:

In [25]:
30 * 10000 / 60 / 60

83.33333333333333

So we're slowing down somewhere, I'm just not sure where.  We probably need to collect some data on the longer running computations to improve the overall approach.  (I don't know of any number theory that might help us determine the digits with fewer computations.)

To start, create a Python decorator that can wrap an arbitrary function to measure how long it takes to complete.  We're also going to build in code to kill the function after a set period of time in order to control the overall run time.

**Note**: this decorator was very useful; it's going in the library.

In [26]:
import signal
import time
from functools import wraps

def timed(timeout_seconds=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if timeout_seconds is not None:  # If timeout is specified
                def timeout_handler(signum, frame):
                    raise TimeoutError("Function execution exceeded the time limit.")
                
                # Set the timeout signal handler
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(timeout_seconds)  # Set the timeout duration
            
            try:
                start_time = time.time()  # Start timing
                result = func(*args, **kwargs)  # Execute the function
                elapsed_time = time.time() - start_time  # Calculate elapsed time
                return result, elapsed_time
            except TimeoutError as e:
                ## print(f"Timeout occurred: {e}")
                return None, None
            finally:
                if timeout_seconds is not None:
                    signal.alarm(0)  # Cancel the alarm
        return wrapper
    return decorator

Functions need to be redefined to make use of the decorator:

In [27]:
@timed(10)
def g(n):
    k = n
    while not digit_greater_than_two(k):
        k += n

    return k

In [28]:
g(89)

(1121222, 0.0020639896392822266)

In [29]:
g(999)

(None, None)

Now we'll do some data analysis to figure out where we're having problems ...

In [30]:
import pandas as pd

In [31]:
# Define the column structure
columns = ['n', 'f(n)', 'Elapsed Time']
data_types = {'n': 'Int64', 'f(n)': 'Int64', 'Elapsed Time': 'Int64'}

# Create an empty DataFrame with specified types
df = pd.DataFrame(columns=columns).astype(data_types)

In [32]:
rows = []

for n in range(1, 10001):
    (f, t) = g(n)
    
    rows.append({'n': n, 'f(n)': f, 'Elapsed Time': t})

df = pd.DataFrame(rows)
df['f(n)'] = df['f(n)'].astype('Int64')
df['Elapsed Time'] = (df['Elapsed Time']).round().astype('Int64')

In [33]:
with pd.option_context('display.max_rows', None):
    print(df[-20:])

          n          f(n)  Elapsed Time
9980   9981     110200221             0
9981   9982      21012110             0
9982   9983     210112201             0
9983   9984     111002112             0
9984   9985     212201220             0
9985   9986     121100222             0
9986   9987      22201101             0
9987   9988    1022012112             0
9988   9989          <NA>          <NA>
9989   9990          <NA>          <NA>
9990   9991      11100001             0
9991   9992      11101112             0
9992   9993     211122111             0
9993   9994    2012122002             0
9994   9995   11110002220             0
9995   9996      22211112             0
9996   9997   20120222122             0
9997   9998  111122211112             2
9998   9999          <NA>          <NA>
9999  10000         10000             0


There are 13 values of $n$ for which computing $f(n)$ takes an extremely long time.  I suspect they're all related.

(They are all multiples of 999, except for 9899, 9989, and 9999.)

In [34]:
df[df['f(n)'].isna()]

,n,f(n),Elapsed Time
998,999,<NA>,<NA>
1997,1998,<NA>,<NA>
2996,2997,<NA>,<NA>
3995,3996,<NA>,<NA>
4994,4995,<NA>,<NA>
5993,5994,<NA>,<NA>
6992,6993,<NA>,<NA>
7991,7992,<NA>,<NA>
8990,8991,<NA>,<NA>
9898,9899,<NA>,<NA>


We need a better approach for testing large multiples, since numbers with lots of $9$s tend to have $9$s, or other large values, in their multiples.  So we'll approach the problem backwards: we'll generate large integers containing only digits in $\{0, 1, 2\}$, and see which ones have a given number as a factor.

This combinatorial approach will still require a lot of processing - if the value of $f(n)$ has m digits, we'll still have to generate

$$
\sum_{k=0}^{m} x^k = {{1-x^{m+1}} \over {1-x}}
$$

for testing.  So if the solution has 20 digits, we'll have to test 5,230,176,601 values.  But this may be less than the number of multiples we would have to test, and so we'd converge on an answer faster (and would improve our odds if the solution has less than 20 digits).

## Generating Values

In [35]:
import itertools

In [36]:
# I've used this function a few times now, so it needs to go into the library as well.
def digits_to_number(t):
    return int(str().join(map(str, t)))

In [37]:
# Test the approach on a large number with lots of 9's
for item in itertools.product('012',repeat=20):
    value = digits_to_number(item)
    if value < 9999:
        continue
    if value % 9999 == 0:
        print(value)
        break

11112222222222222222


In [38]:
11112222222222222222 / 9999

1111333355557778.0

Bingo!  Note that this solution would have required generating 1,111,333,355,557,778 multiples of 9,999.  So the 5,230,176,601 combinations that we tried to determine this value is actally a 99% reduction in computational overhead. :)

In [39]:
5230176601 / 1111333355557778 - 1 * 100

-99.99999529378239

Let's wrap up this approach in a new function:

In [40]:
@timed()
def h(n):
    for item in itertools.product('012',repeat=20):
        value = digits_to_number(item)
        if value < n:
            continue
        if value % n == 0:
            break

    return value

Then repeat our calculation, utilizing $h(n)$ wherever $g(n)$ takes over 10 seconds to compute:

In [41]:
# Define the column structure
columns = ['n', 'f(n)', 'Elapsed Time']
data_types = {'n': 'Int64', 'f(n)': 'Int64', 'Elapsed Time': 'Int64'}

# Create an empty DataFrame with specified types
df = pd.DataFrame(columns=columns).astype(data_types)

In [ ]:
rows = []

for n in range(1, 10001):
    (f, t) = g(n)

    if not f:
        (f, t) = h(n)
    
    rows.append({'n': n, 'f(n)': f, 'Elapsed Time': t})

df = pd.DataFrame(rows)
df['f(n)'] = df['f(n)'].astype('UInt64')  # Larger data type to hold some of the large values we will be generating
df['Elapsed Time'] = (df['Elapsed Time']).round().astype('Int64')

Check that we have computed a value for every $n$:

In [ ]:
df[df['f(n)'].isna()]['n']

Use floor division to compute
$${f(n)} \over {n}$$
We're not losing any precision, since $f(n)$ is a multiple of $n$.  This does avoid convertion to `float`, which preserves precision when computing the sum.
Note that the `pandas` syntax `df['Ratio'] = (df['f(n)'] / df['n']).astype('Int64')` fails - it cannot convert all of the `float` results to `Int64`, necessitating the floor division and data frame iteration approach.

In [ ]:
for x in list(df.itertuples())[:10]:
    print(x.n, x._2, x._2 // x.n)

Final answer:
$$
\sum_{n=1}^{10000} {f(n) \over n} = 1,111,981,904,675,169
$$

In [ ]:
sum([ x._2 // x.n for x in df.itertuples() ])